In [9]:
from bs4 import BeautifulSoup
from selenium import webdriver


def get_table_data(table_data, tables):

    for table in tables:
        table_html = table.get_attribute("outerHTML")
        soup = BeautifulSoup(table_html, "html.parser")
        expression_data = {}

        # Extract expression, description, matches, and non-matches
        expression_data['id'] = len(table_data) + 1 
        details_link = soup.find('a', class_='buttonSmall', href=True)
        if details_link:
            expression_data['details_link'] = "https://regexlib.com/"+details_link['href'].replace('RETester.aspx', 'REDetails.aspx')
        expression_data['expression'] = soup.find('tr', class_='expression').find('div', class_='expressionDiv').text.strip()
        expression_data['description'] = soup.find('tr', class_='description').find('div', class_='overflowFixDiv').text.strip()
        expression_data['matches'] = soup.find('tr', class_='matches').find('div', class_='overflowFixDiv').text.strip()
        expression_data['non_matches'] = soup.find('tr', class_='nonmatches').find('div', class_='overflowFixDiv').text.strip()

        # Append data to the list
        table_data.append(expression_data)

    return table_data

In [10]:
import requests
from selenium.webdriver.common.by import By

table_data = []


for i in range(1, 43):
    driver = webdriver.Chrome()
    url = "https://regexlib.com/Search.aspx?k=&c=-1&m=-1&ps=100&p="+str(i)

    driver.get(url)

    tables = driver.find_elements(By.CSS_SELECTOR, "table.searchResultsTable")
    table_data = get_table_data( table_data, tables)

    print(len(table_data))
    print(table_data[-1])

    driver.close()




100
{'id': 100, 'details_link': 'https://regexlib.com/REDetails.aspx?regexp_id=109', 'expression': '^(0[1-9]|1[0-2])\\/((0[1-9]|2\\d)|3[0-1])\\/(19\\d\\d|200[0-3])$', 'description': 'This expression is an attempt to match the most obvious features of a valid date in American/US format of the form mm/dd/yyyy for any year 1900 through 2003. It can easily be adapted for different date ranges. It matches any date where the day part is any number 01-31, the month part is any number 01-12, and the year part is any number 1900-2003. The only invalid dates that it matches are ones where the day part may be in the range 01-31 but that range might not be valid for the specified month. An example of an invalid date that it would allow to pass through would be 11/31/2002. This date is invalid because 11/31 of any year does not exist, but this expression would allow it pass through since the day part is in the range 01-31.\nThis regular expression has been used successfully in ASP with VBScript usi

In [11]:
print(len(table_data))

4128


In [12]:
import json
with open('./RegexEval_Init.json', 'w') as f:
    json.dump(table_data, f)